In [314]:
import statsmodels.imputation.mice as mice
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse
from scipy.stats import random_correlation
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split 

In [315]:
import pandas as pd
import numpy as np
from scipy.stats import random_correlation

mean = (3,5,1,7,5,2)
np.random.seed(1)
cov = random_correlation.rvs((0.6, 0.2, 1.2, 3, 0.9, 0.1))
norm = np.random.multivariate_normal(mean, cov, 200)
print(norm[:5])

[[1.53077917 5.64589272 0.286451   6.21355467 5.38840393 3.36436987]
 [4.04795328 3.89480148 1.49661444 7.84043937 4.83185913 1.82366514]
 [4.58081713 4.67666697 1.40499161 7.69259925 5.02820536 1.31339449]
 [2.75924213 5.74522538 0.82695585 5.65394027 5.61031322 1.30314724]
 [4.09990273 4.03934275 1.56512268 7.33083761 4.3609962  3.53462144]]


In [316]:
beta = np.array([3,1,-1,6,10,3])
data = pd.DataFrame(norm, columns=['x1','x2','x3','x4','x5','x6'])
y = np.matmul(beta,data.values.transpose())
data.iloc[[1,3,5],1] = None
missing_num = [0,50,50,50,100,100,100]
for i in range(1,6):
    idx = np.random.randint(200,size = missing_num[i])
    data.iloc[idx,i] = None
data['y'] = y
data.head()

,x1,x2,x3,x4,x5,x6,y
0,1.530779,NaN,0.286451,NaN,5.388404,3.364370,111.210256
1,4.047953,NaN,1.496614,7.840439,NaN,1.823665,115.374270
2,4.580817,NaN,NaN,NaN,5.028205,NaN,117.391959
3,2.759242,NaN,0.826956,5.653940,5.610313,1.303147,107.132211
4,4.099903,4.039343,1.565123,7.330838,4.360996,3.534621,112.972780


In [317]:
train, test = train_test_split(data, test_size = 0.3, random_state = 42)
X_train, y_train = train.drop("y", axis = 1), train.y
X_test, y_test = test.drop("y", axis = 1), test.y

In [318]:
X_test.head()

,x1,x2,x3,x4,x5,x6
95,3.907128,4.339186,1.103914,6.712477,NaN,NaN
15,2.054794,5.159811,-0.117361,7.125223,5.138517,1.804979
30,2.821469,4.759206,2.035846,6.555154,6.988172,3.580878
158,3.962876,4.301784,0.603700,NaN,5.253411,1.698566
128,2.713962,6.823018,2.888967,6.659656,NaN,2.174055


In [319]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(imputation_order='ascending', initial_strategy='mean',
                 max_iter=100, n_nearest_features=None,
                 random_state=0, sample_posterior=False, tol=0.001,add_indicator=False)
imp.fit(X_train)


IterativeImputer(add_indicator=False, estimator=None,
                 imputation_order='ascending', initial_strategy='mean',
                 max_iter=100, max_value=None, min_value=None,
                 missing_values=nan, n_nearest_features=None, random_state=0,
                 sample_posterior=False, tol=0.001, verbose=0)

In [320]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse

X_train1 = imp.transform(X_train)
linear_model = sm.OLS(y_train, X_train1)
model_fit = linear_model.fit()
print(model_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.099e+04
Date:                Fri, 06 Sep 2019   Prob (F-statistic):          8.94e-178
Time:                        15:11:04   Log-Likelihood:                -424.04
No. Observations:                 140   AIC:                             860.1
Df Residuals:                     134   BIC:                             877.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.4376      0.889      3.866      0.0

In [321]:
X_test1 = imp.transform(X_test)
y_pred = model_fit.predict(X_test1)
mse(y_pred,y_test)

26.16939335271296

In [322]:
def mf(R, K, max_iter, lr = 0.0002, lambd = 0.02, tol = 0.001):
    N = R.shape[0]
    M = R.shape[1]
    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)
    Q = Q.T
    for it in range(max_iter):
        for i in range(N):
            for j in range(M):
                if (np.isnan(R[i][j])):
                    continue
                else:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + lr*(2*eij*Q[k][j]-lambd*P[i][k])
                        Q[k][j] = Q[k][j] + lr*(2*eij*P[i][k]-lambd*Q[k][j])
        e = 0
        for i in range(N):
            for j in range(M):
                if (np.isnan(R[i][j])):
                    continue
                else:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]),2)
                    for k in range(K):
                        e = e + (lambd/2)*(pow(P[i][k],2) + pow(Q[k][j],2))
        if e<tol:
            break
    return P, Q.T

In [323]:
X = X_train.values
P,Q = mf(X,K=10, max_iter = 1000, tol = 0.00001)
X_train1 = np.matmul(P,Q.T)
print(X[:5])
print(X_train1[:5])

[[ 2.05113697  3.69579991 -1.30923149  6.21546534         nan  2.30381404]
 [ 3.1580459   5.06778403  2.42921362  7.69352712  4.68656935  0.89848375]
 [ 4.5018002   4.43025348  2.5196265   8.41511331         nan         nan]
 [ 3.52535861  4.06587471         nan         nan  4.61587201         nan]
 [ 2.80097337  4.19164254  1.00341119         nan  5.46653134  3.75471429]]
[[ 2.27107492  3.79248937 -0.87301749  5.99417341  3.68730319  2.13099079]
 [ 3.70378226  4.72563809  2.4856811   7.2307544   4.97060237  1.55922663]
 [ 4.53143073  4.52510287  2.51911239  8.31036247  4.5572319   1.10397221]
 [ 3.46338     4.19015574  0.15177294  7.50710809  4.49155485  2.1192753 ]
 [ 3.00791815  4.70133323  0.65117588  7.47149706  5.1856678   3.08695236]]


In [227]:
def mf_P(R, Q, max_iter, lr = 0.0002, lambd = 0.02, tol = 0.001):
    
    N = R.shape[0]
    M = R.shape[1]
    K = Q.shape[1]
    Q = Q.T
    P = np.zeros([N,K])
    
    for it in range(max_iter):
        for i in range(N):
            for j in range(M):
                if (np.isnan(R[i][j])):
                    continue
                else:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + lr*(2*eij*Q[k][j]-lambd*P[i][k])
                        R[i][j] = R[i][j] - lr*(2*eij)
        e = 0
        for i in range(N):
            for j in range(M):
                if (np.isnan(R[i][j])):
                    continue
                else:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]),2)
                    for k in range(K):
                        e = e + (lambd/2)*(pow(P[i][k],2) + pow(Q[k][j],2))
        if e<tol:
            break
    return P

In [228]:
X1 = X_test.values
test_P = mf_P(X1, Q, max_iter = 1000, tol = 0.00001)
X_test1 = np.matmul(test_P,Q.T)
print(X_test.values[:5])
print(X_test1[:5])

[[2.33600843 3.04198138 0.49764914 5.0787087         nan        nan]
 [2.41491706 3.89064785 1.10725407 5.38074201 3.97144049 1.52268048]
 [2.74520479 4.19935284 1.42490319 5.93455161 4.3242948  1.58047506]
 [1.83464772 3.38569544 0.70688598        nan 3.49812008 1.28112144]
 [2.37732313 3.97272315 1.01787146 5.6166538         nan 1.89095044]]
[[2.33476762 3.03889953 0.5006836  5.07262311 3.01413684 1.33792367]
 [2.41176613 3.88521007 1.10542386 5.37438211 3.96725106 1.52160948]
 [2.74179614 4.1920556  1.42075358 5.92738479 4.32130224 1.58042537]
 [1.82489328 3.38373581 0.71104978 4.22975741 3.49279136 1.27634122]
 [2.37516377 3.96817882 1.01529792 5.61078582 3.80973849 1.88493005]]


In [284]:
i=0
X_train2 = X_train.copy()
for col in X_train.columns:
    X_train2.loc[np.isnan(X_train.values[:,i]),col] = X_train1[np.isnan(X_train.values[:,i]),i]
    i = i+1

i=0
X_test2 = X_test.copy()
for col in X_test.columns:
    X_test2.loc[np.isnan(X_test.values[:,i]),col] = X_test1[np.isnan(X_test.values[:,i]),i]
    i = i+1

In [285]:
linear_model2 = sm.OLS(y_train, X_train2)
model_fit2 = linear_model2.fit()
print(model_fit2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     5190.
Date:                Wed, 04 Sep 2019   Prob (F-statistic):          5.08e-156
Time:                        17:31:52   Log-Likelihood:                -476.38
No. Observations:                 140   AIC:                             964.8
Df Residuals:                     134   BIC:                             982.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             5.3630      1.166      4.601      0.0

In [286]:
y_pred2 = model_fit2.predict(X_test2)
mse(y_pred2, y_test)

1423.1411084836577